In [24]:
def open_data_tif(tifmode,filename,v_name,iftime,timename,timestart,timeend,iflon,lonname,iflat,latname,latlow,lattop,lonleft,lonright,latresolution,lonresolution,ifband,iflevel,levelname,level,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no'):
    import rioxarray as rxr
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from netCDF4 import Dataset as net
    import xarray as xr
    from datetime import datetime,timedelta
    from dateutil.relativedelta import relativedelta
    import os
    
    plt.rcParams['font.sans-serif']=['SimHei'] #正常显示中文
    plt.rcParams['axes.unicode_minus']=False #正常显示正负号
    v=None
    levels=None
    times=None
    lat=None
    lon=None
    if tifmode=='one':
        file=rxr.open_rasterio(filename)
        if ifinterpolate == 'yes':
            if np.array(getattr(file,latname))[0]<np.array(getattr(file,latname))[-1]:
                file = file.rio.clip_box(minx=lonleft, miny=latlow, maxx=lonright, maxy=lattop)
                inter = str('file.interp('+latname+'=np.arange('+str(latlow)+','+str(lattop+latresolution)+','+str(latresolution)+'),'+lonname+'=np.arange('+str(lonleft)+','+str(lonright+lonresolution)+','+str(lonresolution)+'))')
            else:
                file = file.rio.clip_box(minx=lonleft, miny=latlow, maxx=lonright, maxy=lattop)
                inter = str('file.interp('+latname+'=np.arange('+str(lattop)+','+str(latlow-latresolution)+','+str(-latresolution)+'),'+lonname+'=np.arange('+str(lonleft)+','+str(lonright+lonresolution)+','+str(lonresolution)+'))')
            files=eval(inter)
            file = files
        if iftime  == 'yes' or iftime == 'self':
            times = np.array(getattr(file,timename))
        if iflon == 'yes':
            lon = np.array(getattr(file,lonname))
        if iflat == 'yes':
            lat = np.array(getattr(file,latname))
        v =file
        if iflevel != 'no':
            levels = np.array(getattr(file,levelname))
    elif tifmode == 'more_time' or tifmode =='more_level':
        direc = os.listdir(filename)
        path = []
        file = []
        v = []
        lat = []
        lon = []
        times = []
        levels = []
        if iftime=='create':
            if len(timestart) == 4 :
                for i in range(len(direc)):
                    times.append(datetime(int(pd.to_datetime(str(timestart)).strftime('%Y')),int(pd.to_datetime(str(timestart)).strftime('%m')),int(pd.to_datetime(str(timestart)).strftime('%d')))+ timespace*i * relativedelta(years=+1))
                    times[i]=pd.to_datetime(str(times[i])).strftime('%Y-%m-%d')
                times = np.array(times,dtype = np.datetime64)
            elif len(timestart) == 7 :
                for i in range(len(direc)):
                    times.append(datetime(int(pd.to_datetime(str(timestart)).strftime('%Y')),int(pd.to_datetime(str(timestart)).strftime('%m')),int(pd.to_datetime(str(timestart)).strftime('%d')))+ timespace*i * relativedelta(months=+1))
                    times[i]=pd.to_datetime(str(times[i])).strftime('%Y-%m-%d')
                times = np.array(times,dtype = np.datetime64)
            elif len(timestart) == 10 :
                for i in range(len(direc)):
                    times.append(datetime(int(pd.to_datetime(str(timestart)).strftime('%Y')),int(pd.to_datetime(str(timestart)).strftime('%m')),int(pd.to_datetime(str(timestart)).strftime('%d')))+ timespace*i * timedelta(days=1))
                    times[i]=pd.to_datetime(str(times[i])).strftime('%Y-%m-%d')
                times = np.array(times,dtype = np.datetime64)
            elif len(timestart) == 13 :
                for i in range(len(direc)):
                    times.append(datetime(int(pd.to_datetime(str(timestart)).strftime('%Y')),int(pd.to_datetime(str(timestart)).strftime('%m')),int(pd.to_datetime(str(timestart)).strftime('%d')),int(pd.to_datetime(str(timestart)).strftime('%H')))+ timespace*i * timedelta(hours=1))
                    times[i]=pd.to_datetime(str(times[i])).strftime('%Y-%m-%d %H:%M:%S')
                times = np.array(times,dtype = np.datetime64)
            elif len(timestart) == 16 :
                for i in range(len(direc)):
                    times.append(datetime(int(pd.to_datetime(str(timestart)).strftime('%Y')),int(pd.to_datetime(str(timestart)).strftime('%m')),int(pd.to_datetime(str(timestart)).strftime('%d')),int(pd.to_datetime(str(timestart)).strftime('%H')),int(pd.to_datetime(str(timestart)).strftime('%M')))+ timespace*i * timedelta(minutes=1))
                    times[i]=pd.to_datetime(str(times[i])).strftime('%Y-%m-%d %H:%M:%S')
                times = np.array(times,dtype = np.datetime64)
            elif len(timestart) == 19 :
                for i in range(len(direc)):
                    times.append(datetime(int(pd.to_datetime(str(timestart)).strftime('%Y')),int(pd.to_datetime(str(timestart)).strftime('%m')),int(pd.to_datetime(str(timestart)).strftime('%d')),int(pd.to_datetime(str(timestart)).strftime('%H')),int(pd.to_datetime(str(timestart)).strftime('%M')),int(pd.to_datetime(str(timestart)).strftime('%S')))+ timespace*i * timedelta(seconds=1))
                    times[i]=pd.to_datetime(str(times[i])).strftime('%Y-%m-%d %H:%M:%S')
                times = np.array(times,dtype = np.datetime64)
        for i in range(len(direc)):
            if filename[-1] == '/':  
                path.append(filename+str(direc[i]))
            else:
                path.append(filename+'/'+str(direc[i]))
            file_xr = rxr.open_rasterio(path[i])
            if ifinterpolate == 'yes':
                inter = str('file_xr.interp('+latname+'=np.arange('+str(latlow)+','+str(lattop)+','+str(latresolution)+'),'+lonname+'=np.arange('+str(lonleft)+','+str(lonright)+','+str(lonresolution)+'))')
                files=eval(inter)
                file_xr = files
            file.append(file_xr)
            if tifmode == 'more_time':
                vs = file[i]
                if iftime =='yes':
                    timelist=np.array(getattr(file[i],timename))
                if i != 0:
                    if iftime =='yes':
                        v=xr.concat([v, vs], dim=timename)
                        times=np.concatenate((times,timelist))
                    elif iftime =='create':
                        vs=eval('vs.expand_dims('+timename+'=[np.array(times[i])])')
                        v=xr.concat([v, vs], dim=timename)
                else:
                    if iftime == 'create':
                        vs=eval('vs.expand_dims('+timename+'=[np.array(times[i])])')
                        v=vs
                    elif iftime == 'yes':
                        v = vs
                        times=timelist
            if tifmode == 'more_level':
                if iflevel == 'create':
                    vs = file[i]
                    levels=level
                elif iflevel == 'yes' or iflevel =='all' or iflevel =='self' or iflevel =='selfchose':
                    vs= file[i]
                    levellist=np.array(getattr(file[i],levelname))      
                if i != 0:
                    if iflevel == 'yes' or iflevel =='all' or iflevel =='self' or iflevel =='selfchose':
                        v = xr.concat([v, vs], dim=levelname)
                        levels=np.concatenate((levels,levellist))
                    elif iflevel =='create':
                        vs=eval('vs.expand_dims('+levelname+'=[np.array(levels[i])])')
                        v = xr.concat([v, vs], dim=levelname)
                else:
                    if iflevel == 'create':
                        vs=eval('vs.expand_dims('+levelname+'=[np.array(levels[i])])')
                        v=vs
                    elif iflevel == 'yes' or iflevel =='all' or iflevel =='self' or iflevel =='selfchose':
                        v=vs
                        levels=levellist
        if tifmode == 'more_time':
            if iflon =='yes':
                lon = getattr(file[0],lonname)
            if iflat =='yes':
                lat = getattr(file[0],latname)
            if iflevel != 'no':
                levels = np.array(getattr(file[0],levelname))
        if tifmode == 'more_level':
            if iflon =='yes':
                lon = getattr(file[0],lonname)
            if iflat =='yes':
                lat = getattr(file[0],latname)
            if iftime != 'no':
                times = np.array(getattr(file[0],timename))
    if iftime =='yes':
        if len(timestart) == 4 :
            for i in range(len(times)):
                if timestart == pd.to_datetime(str(np.array(times[i]))).strftime('%Y'):
                    startpoint = i
                if timeend == pd.to_datetime(str(np.array(times[i]))).strftime('%Y'):
                    endpoint = i
        elif len(timestart) == 7 :
            for i in range(len(times)):
                if timestart == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m'):
                    startpoint = i
                if timeend == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m'):
                    endpoint = i
        elif len(timestart) == 10 :
            for i in range(len(times)):
                if timestart == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d'):
                    startpoint = i
                if timeend == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d'):
                    endpoint = i
        elif len(timestart) == 13 :
            for i in range(len(times)):
                if timestart == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d-%H'):
                    startpoint = i
                if timeend == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d-%H'):
                    endpoint = i
        elif len(timestart) == 16 :
            for i in range(len(times)):
                if timestart == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d-%H-%M'):
                    startpoint = i
                if timeend == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d-%H-%M'):
                    endpoint = i
        elif len(timestart) == 19:
            for i in range(len(times)):
                if timestart == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d-%H-%M-%S'):
                    startpoint = i
                if timeend == pd.to_datetime(str(np.array(times[i]))).strftime('%Y-%m-%d-%H-%M-%S'):
                    endpoint = i
    if iftime=='self':
        for i in range(len(times)):
            if timestart == times[i]:
                startpoint = i
            if timeend == times[i]:
                endpoint = i
    if iftime =='yes' or iftime=='self':
        times = times[startpoint:endpoint+1]
    elif iftime =='create':
        startpoint = 0
        endpoint = times.shape[0]
    if iflat == 'yes':
        if float(lat[0])>float(lat[1]):
            lowpoint = int((np.nanmax(lat)-latlow)/latresolution)
            toppoint = int((np.nanmax(lat)-lattop)/latresolution)
        else:
            lowpoint = int((-np.nanmin(lat)+latlow)/latresolution)
            toppoint = int((-np.nanmin(lat)+lattop)/latresolution)
    if iflon == 'yes':
        leftpoint = int((-np.nanmin(lon)+lonleft)/lonresolution)
        rightpoint = int((-np.nanmin(lon)+lonright)/lonresolution)
    if iflevel == 'yes':
        for i in range(0,len(levels)):
            if int(level) == int(levels[i]):
                levelpoint = i
        if ifband == 'yes':
            if float(lat[0])>float(lat[1]):
                v = v[startpoint:endpoint+1,0,levelpoint,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                v = np.array(v[:,::changeresolution,::changeresolution])
            else:
                v = v[startpoint:endpoint+1,0,levelpoint,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                v = np.array(v[:,::changeresolution,::changeresolution])
        elif ifband ==  'no':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,levelpoint,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,levelpoint,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,levelpoint,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,levelpoint,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,levelpoint,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,levelpoint]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[levelpoint,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[::changeresolution,::changeresolution])
                        else:
                            v = v[levelpoint,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[::changeresolution,::changeresolution])
                    else:
                        v = v[levelpoint,leftpoint:rightpoint+1]
                        v = np.array(v[::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[levelpoint,toppoint:lowpoint+1]
                            v = np.array(v[::changeresolution])
                        else:
                            v = v[levelpoint,lowpoint:toppoint+1]
                            v = np.array(v[::changeresolution])
                    else:
                        v = v[levelpoint]
                        v = np.array(v)
    elif iflevel == 'no':
        if ifband == 'yes':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,0,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,0,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,0,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,0]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[::changeresolution,::changeresolution])
                        else:
                            v = v[0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[::changeresolution,::changeresolution])
                    else:
                        v = v[0,leftpoint:rightpoint+1]
                        v = np.array(v[::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[0,toppoint:lowpoint+1]
                            v = np.array(v[::changeresolution])
                        else:
                            v = v[0,lowpoint:toppoint+1]
                            v = np.array(v[::changeresolution])
                    else:
                        v = None
        elif ifband ==  'no':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[::changeresolution,::changeresolution])
                        else:
                            v = v[lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[::changeresolution,::changeresolution])
                    else:
                        v = v[leftpoint:rightpoint+1]
                        v = np.array(v[::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[toppoint:lowpoint+1]
                            v = np.array(v[::changeresolution])
                        else:
                            v = v[lowpoint:toppoint+1]
                            v = np.array(v[::changeresolution])
                    else:
                        v = None
    elif iflevel == 'all' or iflevel =='create':
        if ifband == 'yes':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,:,0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,:,0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,:,0,leftpoint:rightpoint+1]
                        v = np.array(v[:,:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,:,0,toppoint:lowpoint+1]
                            v = np.array(v[:,:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,:,0,lowpoint:toppoint+1]
                            v = np.array(v[:,:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,:,0]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[:,0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,0,::changeresolution,::changeresolution])
                        else:
                            v = v[:,0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[:,0,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[:,0,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[:,0,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[:,0]
                        v = np.array(v)
        elif ifband == 'no':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,:,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,:,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,:,leftpoint:rightpoint+1]
                        v = np.array(v[:,:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,:,toppoint:lowpoint+1]
                            v = np.array(v[:,:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,:,lowpoint:toppoint+1]
                            v = np.array(v[:,:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,:]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[:,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[:,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[:,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[:,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[:,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[:]
                        v = np.array(v)
    elif iflevel == 'self':
        levelstart = 0
        levelend = 0
        for i in range(len(levels)):
            if int(levels[i]) == level[0]:
                levelstart = i
            if int(levels[i]) == level[1]:
                levelend = i
        if ifband == 'yes':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,levelstart:levelend+1,0,leftpoint:rightpoint+1]
                        v = np.array(v[:,:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,0,toppoint:lowpoint+1]
                            v = np.array(v[:,:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,0,lowpoint:toppoint+1]
                            v = np.array(v[:,:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,levelstart:levelend+1,0]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[levelstart:levelend+1,0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[levelstart:levelend+1,0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[levelstart:levelend+1,0,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[levelstart:levelend+1,0,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[levelstart:levelend+1,0,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[levelstart:levelend+1,0]
                        v = np.array(v)
        elif ifband == 'no':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,levelstart:levelend+1,leftpoint:rightpoint+1]
                        v = np.array(v[:,:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,toppoint:lowpoint+1]
                            v = np.array(v[:,:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,levelstart:levelend+1,lowpoint:toppoint+1]
                            v = np.array(v[:,:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,levelstart:levelend+1]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[levelstart:levelend+1,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[levelstart:levelend+1,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[levelstart:levelend+1,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[levelstart:levelend+1,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[levelstart:levelend+1,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[levelstart:levelend+1]
                        v = np.array(v)
        levels = levels[levelstart:levelend+1]
    elif iflevel == 'selfchose':
        selflevel = []
        j=0
        for i in range(len(levels)):
            if j>= len(level):
                break
            if int(levels[i]) == level[j]:
                selflevel.append(i)
                j=j+1
        if ifband == 'yes':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,selflevel,0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,selflevel,0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,selflevel,0,leftpoint:rightpoint+1]
                        v = np.array(v[:,:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,selflevel,0,toppoint:lowpoint+1]
                            v = np.array(v[:,:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,selflevel,0,lowpoint:toppoint+1]
                            v = np.array(v[:,:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,selflevel,0]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[selflevel,0,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[selflevel,0,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[selflevel,0,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[selflevel,0,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[selflevel,0,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[selflevel,0]
                        v = np.array(v)
        elif ifband == 'no':
            if iftime != 'no':
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,selflevel,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,selflevel,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,:,::changeresolution,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,selflevel,leftpoint:rightpoint+1]
                        v = np.array(v[:,:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[startpoint:endpoint+1,selflevel,toppoint:lowpoint+1]
                            v = np.array(v[:,:,::changeresolution])
                        else:
                            v = v[startpoint:endpoint+1,selflevel,lowpoint:toppoint+1]
                            v = np.array(v[:,:,::changeresolution])
                    else:
                        v = v[startpoint:endpoint+1,selflevel]
                        v = np.array(v)
            else:
                if iflon != 'no':
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[selflevel,toppoint:lowpoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                        else:
                            v = v[selflevel,lowpoint:toppoint+1,leftpoint:rightpoint+1]
                            v = np.array(v[:,::changeresolution,::changeresolution])
                    else:
                        v = v[selflevel,leftpoint:rightpoint+1]
                        v = np.array(v[:,::changeresolution])
                else:
                    if iflat != 'no':
                        if float(lat[0])>float(lat[1]):
                            v = v[selflevel,toppoint:lowpoint+1]
                            v = np.array(v[:,::changeresolution])
                        else:
                            v = v[selflevel,lowpoint:toppoint+1]
                            v = np.array(v[:,::changeresolution])
                    else:
                        v = v[selflevel]
                        v = np.array(v)
        levels = levels[selflevel]
    if iflon !='no':
        lon = lon[leftpoint:rightpoint+1:changeresolution]
    if iflat !='no':
        if float(lat[0])>float(lat[1]):
            lat = lat[toppoint:lowpoint+1:changeresolution]
        else:
            lat = lat[lowpoint:toppoint+1:changeresolution]
    if ifchange_west_east =='yes':
        if np.nanmin(lon)<0:
            right = 360.0 - changeresolution*lonresolution
            if iflevel == 'all' or iflevel == 'self' or iflevel == 'selfchose' or iflevel == 'create':
                if iftime !='no':
                    mid = int(v.shape[3]/2)
                    lon = np.linspace(0.0,right,v.shape[3])
                    vwest = v[:,:,:,0:mid]
                    veast = v[:,:,:,mid:]
                    v = np.concatenate((veast,vwest),axis=3)
                    lonleft = 0.0
                    lonright = right
                else:
                    mid = int(v.shape[2]/2)
                    lon = np.linspace(0.0,right,v.shape[2])
                    vwest = v[:,:,0:mid]
                    veast = v[:,:,mid:]
                    v = np.concatenate((veast,vwest),axis=2)
                    lonleft = 0.0
                    lonright = right
            else:
                if iftime !='no':
                    mid = int(v.shape[2]/2)
                    lon = np.linspace(0.0,right,v.shape[2])
                    vwest = v[:,:,0:mid]
                    veast = v[:,:,mid:]
                    v = np.concatenate((veast,vwest),axis=2)
                    lonleft = 0.0
                    lonright = right
                else:
                    mid = int(v.shape[1]/2)
                    lon = np.linspace(0.0,right,v.shape[1])
                    vwest = v[:,0:mid]
                    veast = v[:,mid:]
                    v = np.concatenate((veast,vwest),axis=1)
                    lonleft = 0.0
                    lonright = right
        else:
            right = 180.0 - changeresolution*lonresolution
            if iflevel == 'all' or iflevel == 'self' or iflevel == 'selfchose' or iflevel =='create':
                if iftime !='no':
                    mid = int(v.shape[3]/2)
                    lon = np.linspace(-180.0,right,v.shape[3])
                    veast = v[:,:,:,0:mid]
                    vwest = v[:,:,:,mid:]
                    v = np.concatenate((vwest,veast),axis=3)
                    lonleft = -180.0
                    lonright = right
                else:
                    mid = int(v.shape[2]/2)
                    lon = np.linspace(-180.0,right,v.shape[2])
                    veast = v[:,:,0:mid]
                    vwest = v[:,:,mid:]
                    v = np.concatenate((vwest,veast),axis=2)
                    lonleft = -180.0
                    lonright = right
            else:
                if iftime !='no':
                    mid = int(v.shape[2]/2)
                    lon = np.linspace(-180.0,right,v.shape[2])
                    veast = v[:,:,0:mid]
                    vwest = v[:,:,mid:]
                    v = np.concatenate((vwest,veast),axis=2)
                    lonleft = -180.0
                    lonright = right
                else:
                    mid = int(v.shape[1]/2)
                    lon = np.linspace(-180.0,right,v.shape[1])
                    veast = v[:,0:mid]
                    vwest = v[:,mid:]
                    v = np.concatenate((vwest,veast),axis=1)
                    lonleft = -180.0
                    lonright = right
    v=np.array(v)
    lat=np.array(lat)
    lon=np.array(lon)
    levels=np.array(levels)
    times=np.array(times)
    if iflevel == 'all' or iflevel == 'self' or iflevel == 'selfchose' or iflevel =='create':
        if iftime !='no':
            if iflat !='no':
                if iflon !='no':
                    v = xr.DataArray(v, [(timename,times),(levelname,levels),(latname,lat),(lonname,lon)])
                else:
                    v = xr.DataArray(v, [(timename,times),(levelname,levels),(latname,lat)])
            else:
                if iflon !='no':
                    v = xr.DataArray(v, [(timename,times),(levelname,levels),(lonname,lon)])
                else:
                    v = xr.DataArray(v, [(timename,times),(levelname,levels)])
        else:
            if iflat !='no':
                if iflon !='no':
                    v = xr.DataArray(v, [(levelname,levels),(latname,lat),(lonname,lon)])
                else:
                    v = xr.DataArray(v, [(levelname,levels),(latname,lat)])
            else:
                if iflon !='no':
                    v = xr.DataArray(v, [(levelname,levels),(lonname,lon)])
                else:
                    v = xr.DataArray(v, [(levelname,levels)])
        levels = v[levelname]
    else:
        if iftime !='no':
            if iflat !='no':
                if iflon !='no':
                    v = xr.DataArray(v, [(timename,times),(latname,lat),(lonname,lon)])
                else:
                    v = xr.DataArray(v, [(timename,times),(latname,lat)])
            else:
                if iflon !='no':
                    v = xr.DataArray(v, [(timename,times),(lonname,lon)])
                else:
                    v = xr.DataArray(v, [(timename,times)])
        else:
            if iflat !='no':
                if iflon !='no':
                    v = xr.DataArray(v, [(latname,lat),(lonname,lon)])
                else:
                    v = xr.DataArray(v, [(latname,lat)])
            else:
                if iflon !='no':
                    v = xr.DataArray(v, [(lonname,lon)])
                else:
                    v = None
        levels = None
    if iftime !='no':
        times = v[timename]
    else:
        times = None
    if iflon !='no':
        lon = v[lonname]
    else:
        lon = None
    if iflat !='no':
        lat = v[latname]
    else:
        lat = None
    return v,lon,lat,levels,latlow,lattop,lonleft,lonright,times


In [25]:
v,lon,lat,levels,latlow,lattop,lonleft,lonright,times=open_data_tif('more_time','E:/9_tif_qzgy_recut/9_tif_qzgy_recut/','pre','create','time','2001-01-01','2020-12-31','yes','x','yes','y',25.88,39.88,73.38,104.6,0.25,0.25,'yes','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')

In [28]:
import Auto_paint_self
Auto_paint_self.create_nc(times,lat,lon,None,v,'time','lat','lon','pre','no',None,'yes','yes','yes','E:/qzgy_pre.nc')